### Large Language Models

| Model              | `llm_commons.proxy`                                   | `llm_commons.langchain.proxy`             |
|--------------------|---------------------------------------------------------|---------------------------------------------|
| `text-davinci-003` | `.openai.Completion(deployment_id='text-davinci-003')` | `.OpenAI(deployment_id='text-davinci-003')` |
| `gpt-35-turbo` | `.openai.ChatCompletion(deployment_id='gpt-35-turbo')` | `.ChatOpenAI(deployment_id='gpt-35-turbo')` |
| `gpt-4` | `.openai.ChatCompletion(deployment_id='gpt-4')` | `.ChatOpenAI(deployment_id='gpt-4')` |
| `gpt-4-32k` | `.openai.ChatCompletion(deployment_id='gpt-4-32k')` | `.ChatOpenAI(deployment_id='gpt-4-32k')` |
| `alephalpha` | `.aleph_alpha.Client(deployment_id='alephalpha')` | `.AlephAlpha(deployment_id='alephalpha')` |
| `anthropic-claude-v1` |  | `.BedrockChat(deployment_id='anthropic-claude-v1')` |
| `anthropic-claude-v2` |  | `.BedrockChat(deployment_id='anthropic-claude-v2')` |
| `anthropic-claude-instant-v1` |  | `.BedrockChat(deployment_id='anthropic-claude-instant-v1')` |
| `anthropic-claude-v1-100k` |  | `.BedrockChat(deployment_id='anthropic-claude-v1-100k')` |
| `anthropic-claude-v2-100k` |  | `.BedrockChat(deployment_id='anthropic-claude-v2-100k')` |
| `anthropic-direct-claude-instant-1` |  | `.BedrockChat(deployment_id='anthropic-direct', model_kwargs={'model': 'claude-instant-1'})` |
| `anthropic-direct-claude-2` |  | `.BedrockChat(deployment_id='anthropic-direct', model_kwargs={'model': 'claude-2'})` |
| `ai21-j2-grande-instruct` |  | `.Bedrock(deployment_id='ai21-j2-grande-instruct')` |
| `ai21-j2-jumbo-instruct` |  | `.Bedrock(deployment_id='ai21-j2-jumbo-instruct')` |
| `amazon-titan-tg1-large` |  | `.Bedrock(deployment_id='amazon-titan-tg1-large')` |
| `gcp-text-bison-001` |  | `.GooglePalm(deployment_id='gcp-text-bison-001')` |
| `falcon-7b` | `.huggingface.Client(deployment_id='falcon-7b')` | `.HuggingFaceTextGenInference(deployment_id='falcon-7b')` |
| `falcon-40b-instruct` | `.huggingface.Client(deployment_id='falcon-40b-instruct')` | `.HuggingFaceTextGenInference(deployment_id='falcon-40b-instruct')` |
| `llama2-13b-chat-hf` | `.huggingface.Client(deployment_id='llama2-13b-chat-hf')` | `.HuggingFaceTextGenInference(deployment_id='llama2-13b-chat-hf')` |

### Embedding Models

| Model              | `llm_commons.proxy`                                   | `llm_commons.langchain.proxy`             |
|--------------------|---------------------------------------------------------|---------------------------------------------|
| `text-embedding-ada-002-v2` | `.openai.Embedding(deployment_id='text-embedding-ada-002-v2')` | `.OpenAIEmbeddings(deployment_id='text-embedding-ada-002-v2')` |
| `amazon-titan-e1t-medium` |  | `.BedrockEmbeddings(deployment_id='amazon-titan-e1t-medium')` |
| `gcp-textembedding-gecko-001` |  | `.GoogleEmbeddings(deployment_id='gcp-textembedding-gecko-001')` |

## Next steps
#### ~Make this work with PDF input + custom question~
#### ~Iteratively try all LLMs~
#### ~Configure LLM-specific embeddings~
#### ~Build excel export~
#### ~Should cover multiple iterations~
#### K
#### ~chunking~
#### ~max_tokens~
#### ~temperature~
#### ~Put this in a diagram~


#### More rows with accuracy measures; ragas and ROGUE

#### Note that `from llm_commons.langchain.btp_llm import BTPOpenAI, BTPOpenAIEmbeddings` are deprecated in 0.2.0 and `calm-converse-gaihsdk` environment needs the keyconfig setup done to be able to use this

In [1]:
!ls config

key-genaihub.json key.json


In [ ]:
!btp_llm configure -k config/key.json

/Users/I516992/anaconda3/envs/calm-converse-gaihsdk/bin/btp_llm:5: UserWarning: Starting from verison 1.0.0 the default proxy_version was set to 'gen-ai-hub'. Use gen_ai_hub.proxy.core.proxy_clients.set_proxy_version('btp') to set the proxy_version to 'btp'.
  from llm_commons.btp_llm.cli import cli
Creating new config /Users/I516992/.btp_llm/config.json
A config file for the default profile already exists. To do want to replace it? [y/N]: 

In [1]:
!pip install "sap-llm-commons[all]"==0.1.8

Looking in indexes: https://pypi.org/simple, https://int.repositories.cloud.sap/artifactory/api/pypi/deploy-releases-pypi/simple, https://int.repositories.cloud.sap/artifactory/api/pypi/proxy-deploy-releases-hyperspace-pypi/simple


In [17]:
!python --version

Python 3.11.7


In [1]:
from langchain.document_loaders import TextLoader
from PyPDF2 import PdfReader

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback

In [3]:
from llm_commons.langchain.proxy.openai import ChatOpenAI
from llm_commons.langchain.proxy.bedrock import BedrockChat, Bedrock
from llm_commons.langchain.proxy.google import GooglePalm
from llm_commons.langchain.proxy.huggingface import HuggingFaceTextGenInference
from llm_commons.langchain.proxy.aleph_alpha import AlephAlpha

from llm_commons.langchain.proxy.openai import OpenAI
from llm_commons.langchain.proxy import OpenAIEmbeddings

from llm_commons.langchain.proxy.bedrock import BedrockEmbeddings
from llm_commons.langchain.proxy.google import GoogleEmbeddings

In [4]:
import openai
import time

In [5]:
import pandas as pd

In [6]:
from rouge_score import rouge_scorer

In [7]:
max_tokens = 800
temperature = 0
temperature_hf = 0.1

In [8]:
all_llms = {
    'text-davinci-003': OpenAI(deployment_id='text-davinci-003', max_tokens = max_tokens, temperature = temperature),
    'gpt-35-turbo':  ChatOpenAI(deployment_id='gpt-35-turbo', max_tokens = max_tokens, temperature = temperature),
    'gpt-4':  ChatOpenAI(deployment_id='gpt-4', max_tokens = max_tokens, temperature = temperature),
    'gpt-4-32k':  ChatOpenAI(deployment_id='gpt-4-32k', max_tokens = max_tokens, temperature = temperature),
    # 'alephalpha':  AlephAlpha(deployment_id='alephalpha', maximum_tokens=max_tokens, temperature=temperature),
    'anthropic-claude-v1':  BedrockChat(deployment_id='anthropic-claude-v1', model_kwargs={"temperature": temperature, "max_tokens_to_sample": max_tokens}),
    'anthropic-claude-v2':  BedrockChat(deployment_id='anthropic-claude-v2', model_kwargs={"temperature": temperature, "max_tokens_to_sample": max_tokens}),
    'anthropic-claude-instant-v1':  BedrockChat(deployment_id='anthropic-claude-instant-v1', model_kwargs={"temperature": temperature, "max_tokens_to_sample": max_tokens}),
    'anthropic-claude-v1-100k':  BedrockChat(deployment_id='anthropic-claude-v1-100k', model_kwargs={"temperature": temperature, "max_tokens_to_sample": max_tokens}),
    'anthropic-claude-v2-100k':  BedrockChat(deployment_id='anthropic-claude-v2-100k', model_kwargs={"temperature": temperature, "max_tokens_to_sample": max_tokens}),
    'anthropic-direct-claude-instant-1':  BedrockChat(deployment_id='anthropic-direct', model_kwargs={'model': 'claude-instant-1', "temperature": temperature, "max_tokens_to_sample": max_tokens}),
    'anthropic-direct-claude-2':  BedrockChat(deployment_id='anthropic-direct', model_kwargs={'model': 'claude-2', "temperature": temperature, "max_tokens_to_sample": max_tokens}),
    'ai21-j2-grande-instruct':  Bedrock(deployment_id='ai21-j2-grande-instruct', model_kwargs={"temperature": temperature, "maxTokens": max_tokens}),
    'ai21-j2-jumbo-instruct':  Bedrock(deployment_id='ai21-j2-jumbo-instruct', model_kwargs={"temperature": temperature, "maxTokens": max_tokens}),
    'amazon-titan-tg1-large':  Bedrock(deployment_id='amazon-titan-tg1-large', model_kwargs={"temperature": temperature}),
    'gcp-text-bison-001':  GooglePalm(deployment_id='gcp-text-bison-001', temperature = temperature, max_output_tokens = max_tokens),
    'falcon-7b':  HuggingFaceTextGenInference(deployment_id='falcon-7b'),#, temperature = temperature, max_new_tokens = max_tokens), # issue https://github.tools.sap/AI-Playground-Projects/llm-commons/issues/106
    'falcon-40b-instruct':  HuggingFaceTextGenInference(deployment_id='falcon-40b-instruct', temperature = temperature_hf, max_new_tokens = max_tokens),
    'llama2-13b-chat-hf':  HuggingFaceTextGenInference(deployment_id='llama2-13b-chat-hf'),#, temperature = temperature, max_new_tokens = max_tokens),
}

/Users/I516992/anaconda3/envs/calm-converse-gaihsdk/lib/python3.11/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
all_embeddings = {
    'text-embedding-ada-002-v2':  OpenAIEmbeddings(deployment_id='text-embedding-ada-002-v2'),
    'amazon-titan-e1t-medium':  BedrockEmbeddings(deployment_id='amazon-titan-e1t-medium'),
    'gcp-textembedding-gecko-001':  GoogleEmbeddings(deployment_id='gcp-textembedding-gecko-001'),
}

In [10]:
open_ai_models = set([
    'text-davinci-003',
    'gpt-35-turbo',
    'gpt-4',
    'gpt-4-32k',
    'alephalpha'
])

bedrock_models = set([
    'anthropic-claude-v1',
    'anthropic-claude-v2',
    'anthropic-claude-instant-v1',
    'anthropic-claude-v1-100k',
    'anthropic-claude-v2-100k',
    'anthropic-direct-claude-instant-1',
    'anthropic-direct-claude-2',
    'ai21-j2-grande-instruct',
    'ai21-j2-jumbo-instruct',
    'amazon-titan-tg1-large'
])

gcp_models = set([
    'gcp-text-bison-001'
])

huggingfacemodels = set([
    'falcon-7b',
    'falcon-40b-instruct',
    'llama2-13b-chat-hf'
])

In [11]:
def get_embedding(llm):
    return all_embeddings['text-embedding-ada-002-v2']
    if llm in open_ai_models:
        return all_embeddings['text-embedding-ada-002-v2']
    if llm in bedrock_models:
        return all_embeddings['amazon-titan-e1t-medium']
    else:
        return all_embeddings['gcp-textembedding-gecko-001']

#### #TODO implement try catch for ratelimit error

In [12]:
# pdf version

# pdf_reader = PdfReader('corpus/pdf/sf/Integrating SAP SuccessFactors Employee Central with SAP Master Data Integration.pdf')
pdf_reader = PdfReader('corpus/pdf/iep/IEP_help_and_support.pdf')
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

Subject: Integration Issues with CALM and SNOW for Event Management

Error Details:
- CALM and SNOW integration does not show correlation for the same events at CALM side.
- Multiple events from Cloud ALM are creating separate incidents with different IDs in SNOW for the same issue.
- Bad requests are being seen at the External API console in ALM.

Responses:
- The customer is requesting assistance in fixing both issues and how to consolidate events in ALM to avoid repeated SNOW events.
- The customer suspects that the issue may be related to how the Health Monitoring app triggers the logic to call the API or the flow controlled on the External API Management side.
- The customer has provided additional details, including attachments, to investigate the issue further.

Solutions:
- Additional details need to be gathered from the SAP Cloud ALM tenant, including screenshots of API calls in the External API Management app and alerts in the Operations app.
- The issue may be related to the response mapping in the custom mapping provided by the customer. The mapping needs to be changed for Event Manager integration.
- The customer's scenario does not fulfill the requirements for simple integrations, and a complex integration involving integration partners may be necessary.
- Support for complex integrations is provided by the integration partner.

Please provide the requested additional details for further investigation and resolution of the issues.

In [13]:
query = """
As a Level 1 Support Engineer, your responsibility entails resolving the encountered issue effectively. Your primary objective is to pinpoint and rectify the error, thus facilitating uninterrupted integration and system functionality.
Moreover, ensure thoroughness by citing pertinent page numbers from the accompanying documentation.

Subject: Integration Issues with CALM and SNOW for Event Management

Error Details:
- CALM and SNOW integration does not show correlation for the same events at CALM side.
- Multiple events from Cloud ALM are creating separate incidents with different IDs in SNOW for the same issue.
- Bad requests are being seen at the External API console in ALM.
"""

truth = """
Solutions:

Additional details need to be gathered from the SAP Cloud ALM tenant, including screenshots of API calls in the External API Management app and alerts in the Operations app.
The issue may be related to the response mapping in the custom mapping provided by the customer. The mapping needs to be changed for Event Manager integration.
The customer's scenario does not fulfill the requirements for simple integrations, and a complex integration involving integration partners may be necessary.
Support for complex integrations is provided by the integration partner.
"""

generation_count = 3

## Fixed params, varying models

In [14]:
generations = []

for i in range(generation_count):
    
    print("going for generation", str(i))
    
    results = dict()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text=text)

    for llm in all_llms:
    
        embeddings = get_embedding(llm)
        VectorStore = FAISS.from_texts(chunks, embedding=embeddings)
        docs = VectorStore.similarity_search(query=query, k=5)
        chain = load_qa_chain(llm=all_llms[llm], chain_type="stuff")
        
        result = 'error'
        print(llm)
        try: 
            with get_openai_callback() as cb:
                response = chain.run(input_documents=docs, question=query)
                # print(cb)
            print(response)
            result = response
        except RuntimeError as re:
            print("threw error", re)
        except KeyError as ke:
            print("threw error", ke)
        except ValueError as ve:
            print("threw error", ve)
        results[llm] = result
        
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
        scores = scorer.score(truth, result)
        results[llm + '_rouge1_precision'] = scores['rouge1'].precision
        results[llm + '_rouge1_recall'] = scores['rouge1'].recall
        results[llm + '_rouge1_fmeasure'] = scores['rouge1'].fmeasure
        results[llm + '_rougeL_precision'] = scores['rougeL'].precision
        results[llm + '_rougeL_recall'] = scores['rougeL'].recall
        results[llm + '_rougeL_fmeasure'] = scores['rougeL'].fmeasure
    
    generations.append(results)

going for generation 0
text-davinci-003

To resolve this issue, you should first check the mapping file in SAP Cloud ALM to ensure that the available values of the event situation outbound integration are correctly mapped to the API definition of the target system. If the mapping file is not sufficient, you should configure the connectivity in Landscape Management in SAP Cloud ALM and create a mapping file. Additionally, you should check the return values of the integration to ensure that an externalId and externalNumber of the created ticket are being returned. Finally, you should check the External API Management application console for any errors that may have been triggered for the event situation.
gpt-35-turbo
Based on the provided error details, it seems that there are issues with the integration between CALM (SAP Cloud ALM) and SNOW (ServiceNow) for event management. Here are some possible causes and solutions:

1. Correlation Issue: The integration does not show correlation for

anthropic-claude-v1-100k
 I apologize, but I do not have enough context or information to determine the specific cause of the integration issues you are seeing between SAP Cloud ALM and ServiceNow. Without access to the systems and configurations involved, I cannot pinpoint the exact errors or provide a resolution.

The documentation mentions that for ticket integrations, there must be a 1:1 mapping between an event situation and a ticket, and the ticket number must be stored in SAP Cloud ALM. It seems this 1:1 mapping is not occurring properly based on your description. Some possibilities could be:

1) The mapping file in SAP Cloud ALM needs to be adjusted to properly map the event data to the ServiceNow API. Refer to page 6 of the documentation.

2) There is an issue with the destination configuration to ServiceNow, such as incorrect authentication information. Refer to page 8 of the documentation for configuring the destination.

3) Additional integration logic needs to be built usi

gcp-text-bison-001
The first step is to check the event situation in the action log. If the event situation is not available, then the configuration of the event must be checked in the corresponding monitoring application inside of SAP Cloud ALM. If the event is correctly setup, then it is also required that the defined threshold criteria's matched to raise a corresponding event situation.

If the event situation is available, then it is possible to further drill-down to the action log for this event. Here it is possible to see each action which was triggered for the event situation with a short success or error message. In case there is an error raised for the create ticket action, the error text from the target API is directly shown. Further details are available in the console of the External API Management application.

The return values of the integration must return an externalId and externalNumber of the created ticket. During runtime these values are stored in SAP Cloud ALM to 

anthropic-claude-instant-v1
 Based on the context provided, here are the steps I would take to resolve the integration issues between CALM and SNOW:

1. Check the mapping configuration between CALM event situations and SNOW incident fields. The mapping file in CALM needs to correctly map the necessary fields to populate common identifiers like title, description, etc. in SNOW (see pages 12-14). 

2. Verify that the 'externalId' and 'externalNumber' are being returned correctly from SNOW for each event and stored back in CALM. This allows correlating the same issue in both systems (see page 10).

3. Review the error messages at the External API console in CALM to understand what requests are failing. Common issues could be invalid credentials, incorrect API endpoints, missing required fields in requests etc. (see page 11). 

4. Ensure only one CALM event is being processed for the same underlying issue by adding logic to deduplicate events based on common attributes before creating mult

amazon-titan-tg1-large

To resolve the integration issues between CALM and SNOW, follow these steps:
1. Check the Event Mapping in ALM: Ensure that the event mapping in ALM is correctly configured to map the appropriate event fields to the corresponding fields in SNOW.
2. Review the Integration Configuration: Verify that the integration between ALM and SNOW is set up correctly, including the authentication credentials and API endpoints.
3. Check the Threshold Criteria: Ensure that the threshold criteria defined in ALM for raising incidents are met, and that the events are triggering the expected incident creation in SNOW.
4. Review the Action
gcp-text-bison-001
The first step is to check the event situation in the event situation list. If the event situation is not available, then the configuration of the event must be checked in the corresponding monitoring application inside of SAP Cloud ALM. If the event is correctly setup, then it is also required that the defined threshold criteri

anthropic-claude-instant-v1
 Based on the context provided, here are the steps I would take to resolve the integration issues between CALM and SNOW:

1. Check that the connectivity to SNOW is configured correctly in the Landscape Management section of CALM. Refer to page 12 of the documentation for details on "Configure Integration with ServiceNow Incident Management". 

2. Verify that the mapping between the CALM event situation and SNOW incident fields is properly defined. See page 15 for information on creating the mapping. The mapping file format is also documented to ensure the correct values are being passed.

3. Review the external API definition in CALM by checking the webhook and subscription as described on page 16. Make sure the "External Resource Type" is set to "Incident" and the mapping of CALM events to the SNOW resource type is correct. 

4. Examine the error logs in the External API console in CALM as mentioned on page 13. The error text may provide clues on what is ca

amazon-titan-tg1-large

To resolve the integration issues between CALM and SNOW, follow these steps:
1. Check the Event Mapping in ALM: Ensure that the event mapping in ALM is correctly configured to map the appropriate event fields to the corresponding fields in SNOW.
2. Review the Integration Configuration: Verify the integration configuration in ALM, including the API credentials, destination settings, and mapping file.
3. Check the Event Correlation: Confirm that the events in CALM are correctly correlated with the events in SNOW to ensure proper mapping.
4. Resolve API Errors: Address any API errors or issues that are
gcp-text-bison-001
The first step is to check the event situation in the action log. If the event situation is not available, then the configuration of the event must be checked in the corresponding monitoring application inside of SAP Cloud ALM. If the event is correctly setup, then it is also required that the defined threshold criteria's matched to raise a corresp

In [15]:
pd.DataFrame(generations).to_csv('results/iep_CS20230006563076_all_models_rouge.csv')

## Fixed model, changing params, ratelimiterror fixed at both embeddings & llm levels
#### *fixed* ~but it's probably wrong because it calls the entire method all over again, i need to put the core out into a different module~
source: https://github.com/langchain-ai/langchain/discussions/8922

In [17]:
import time
import random

def embeddings_with_retry(chunks, embedding, max_retries=10):
    for i in range(max_retries):
        try:
            return FAISS.from_texts(chunks, embedding=embeddings)

        except openai.error.APIError as apie:
            print("ratelimit error, retrying", i, apie)
            wait_time = (2 ** i) + random.random()
            time.sleep(wait_time)

        except RateLimitError:
            print("ratelimit error, retrying", i, apie)
            wait_time = (2 ** i) + random.random()
            time.sleep(wait_time)
            
    raise Exception("Still hitting rate limit after max retries")
    
    
def llm_with_retry(docs, query, max_retries=10):
    for i in range(max_retries):
        try:
            with get_openai_callback() as cb:
                response = chain.run(input_documents=docs, question=query)
                # print(cb)
            print(response)
            return response
        
        except openai.error.APIError as apie:
            print("ratelimit error, retrying", i, apie)
            wait_time = (2 ** i) + random.random()
            time.sleep(wait_time)
            
        except RuntimeError as re:
            print("threw runtime error, retrying", re)
        except KeyError as ke:
            print("threw key error, retrying", ke)
        except ValueError as ve:
            print("threw value error, retrying", ve)
        except APIError as apie:
            print("threw API error, retrying", apie)
            
    raise Exception("Still hitting rate limit after max retries")

### integrate cost into our evaluations as an added dimension
### context-aware chunking based on document sections
### incident solution matching (Jens)

#### KBA in SSDL objectstore can be integrated
#### route vector engine through Chris Brahnner's team
### wait for it to be centrally available

In [19]:
llm = all_llms['gpt-35-turbo']

chunk_sizes = [200, 400, 600, 800, 1000, 1200, 1400]
chunk_overlaps = [20, 40, 60, 100, 150, 200, 300, 400, 500, 1000]

generations = []

for i in range(generation_count):

    print("going for generation", str(i))

    results = dict()

    for chunk_size in chunk_sizes:
        for chunk_overlap in chunk_overlaps:

            if chunk_size < chunk_overlap:
                continue

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                length_function=len
            )
            chunks = text_splitter.split_text(text=text)

            params = "cs_" + str(chunk_size) + "|co_" + str(chunk_overlap)
            print(params)

            
            embeddings = get_embedding(llm)
            VectorStore = embeddings_with_retry(chunks, embeddings, max_retries=10)
            docs = VectorStore.similarity_search(query=query, k=5)
            chain = load_qa_chain(llm=llm, chain_type="stuff")
            
            result = 'error'    
            result = llm_with_retry(docs, query, max_retries=10)
            results[params] = result

            scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
            scores = scorer.score(truth, result)

            results[params + '_rouge1_precision'] = scores['rouge1'].precision
            results[params + '_rouge1_recall'] = scores['rouge1'].recall
            results[params + '_rouge1_fmeasure'] = scores['rouge1'].fmeasure
            results[params + '_rougeL_precision'] = scores['rougeL'].precision
            results[params + '_rougeL_recall'] = scores['rougeL'].recall
            results[params + '_rougeL_fmeasure'] = scores['rougeL'].fmeasure

    generations.append(results)

going for generation 0
cs_200|co_20


ratelimit error, retrying 0 Invalid response object from API: '{"data":{"error":"HTTPError: {\\"error\\":{\\"code\\":\\"429\\",\\"message\\": \\"Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 26 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\\"}}","status_code":429},"status":429}' (HTTP response code was 500)


ratelimit error, retrying 1 Invalid response object from API: '{"data":{"error":"HTTPError: {\\"error\\":{\\"code\\":\\"429\\",\\"message\\": \\"Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 20 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\\"}}","status_code":429},"status":429}' (HTTP response code was 500)


ratelimit error, retrying 2 Invalid response object from API: '{"data":{"error":"HTTPError: {\\"error\\":{\\"code\\":\\"429\\",\\"message\\": \\"Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 12 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\\"}}","status_code":429},"status":429}' (HTTP response code was 500)


ratelimit error, retrying 3 Invalid response object from API: '{"data":{"error":"HTTPError: {\\"error\\":{\\"code\\":\\"429\\",\\"message\\": \\"Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 3 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\\"}}","status_code":429},"status":429}' (HTTP response code was 500)
Based on the provided context, it seems that the error is related to connecting to a destination server. The error message suggests checking if the IP address mentioned in the provided URL is allowed by the destination server. It also mentions contacting your IT for more information.

To resolve this error, you can follow these steps:

1. Check the IP address mentioned in the URL: https://launchpad.support.sap.com/#/notes/2395508. Make sure it is allowed by the destination server. You

...

The error message suggests that there is an issue with connecting to the destination server. Here are the steps you can take to resolve the error:

1. Check if the IP address mentioned in the provided URL (https://launchpad.support.sap.com/#/notes/2395508) is allowed by your destination server. Contact your IT department to verify if the IP address is whitelisted.

2. Make sure that the URL mentioned in the error message is correct and accessible. In this case, the URL is 'https//.../sap.odm.finance.costobject.costcenter'. Check if there are any typos or missing characters in the URL.

3. Verify if the destination server is up and running. You can try accessing the URL in a web browser to see if it returns a valid response.

4. If the issue persists, contact your IT department or the support team for the destination server for further assistance. They will be able to provide more information and help resolve the error.
cs_1400|co_400


The error message suggests that there is an issue with connecting to the destination server. Here are the steps you can take to resolve the error:

1. Check if the IP address mentioned in the URL 'https://launchpad.support.sap.com/#/notes/2395508' is allowed by your destination server. Contact your IT department to verify if the IP address is whitelisted.

2. Make sure that the URL mentioned in the error message is correct and accessible. In this case, the URL is 'https//.../sap.odm.finance.costobject.costcenter'. Check if there are any typos or missing characters in the URL.

3. Verify if the destination server is up and running. You can try accessing the URL in a web browser to see if it returns a valid response.

4. If the destination server is on-premise, ensure that the Cloud Connector is properly configured and running. The Cloud Connector is responsible for establishing a secure connection between the cloud and on-premise systems.

5. If the issue persists, contact your IT depar

The error message suggests that there is an issue with connecting to the destination server. Here are the steps you can take to resolve the error:

1. Check the IP Address: Verify if the IP Address mentioned in the provided URL (https://launchpad.support.sap.com/#/notes/2395508) is allowed by your destination server. Contact your IT department to ensure that the IP Address is whitelisted.

2. Verify the URL: Double-check the URL mentioned in the error message ('https//.../sap.odm.finance.costobject.costcenter'). Make sure that the URL is correct and complete. It should include the necessary information to connect to the destination server.

3. Check the Response Code: The error message indicates that a 404 HTTP response code was received. This typically means that the requested resource was not found on the server. Verify if the URL is correct and that the resource exists on the destination server.

4. Contact Support: If the issue persists, it is recommended to contact SAP support for

ratelimit error, retrying 0 Invalid response object from API: '{"data":{"error":"HTTPError: {\\"error\\":{\\"code\\":\\"429\\",\\"message\\": \\"Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.\\"}}","status_code":429},"status":429}' (HTTP response code was 500)


The error message suggests that there is an issue with connecting to the destination server. Here are the steps you can take to resolve the error:

1. Check if the IP address mentioned in the provided URL (https://launchpad.support.sap.com/#/notes/2395508) is allowed by your destination server. Contact your IT department to verify if the IP address is whitelisted or if any network restrictions are in place.

2. Ensure that the URL mentioned in the error message ('https//.../sap.odm.finance.costobject.costcenter') is correct and accessible. Double-check the URL for any typos or missing characters. If necessary, consult the documentation or contact the system administrator for the correct URL.

3. Verify if the destination server is up and running. You can try accessing the URL in a web browser to see if it returns a valid response. If the server is not accessible, contact the system administrator to resolve the issue.

4. If the destination server is accessible and the URL is correct, b

## Found best chunk_size and chunk_overlap. Trying k = [1, 15]

In [33]:
llm = all_llms['gpt-35-turbo']

K = list(range(1, 16))

generations = []

for i in range(generation_count):

    print("going for generation", str(i))

    results = dict()

    for k in K:

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=600,
            chunk_overlap=60,
            length_function=len
        )
        chunks = text_splitter.split_text(text=text)

        params = "k_" + str(k)
        print(params)

        embeddings = get_embedding(llm)
        VectorStore = embeddings_with_retry(chunks, embeddings, max_retries=10)
        docs = VectorStore.similarity_search(query=query, k=k)
        chain = load_qa_chain(llm=llm, chain_type="stuff")

        result = 'error'
        result = llm_with_retry(docs, query, max_retries=10)
        results[params] = result

        scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
        scores = scorer.score(truth, result)

        results[params + '_rouge1_precision'] = scores['rouge1'].precision
        results[params + '_rouge1_recall'] = scores['rouge1'].recall
        results[params + '_rouge1_fmeasure'] = scores['rouge1'].fmeasure
        results[params + '_rougeL_precision'] = scores['rougeL'].precision
        results[params + '_rougeL_recall'] = scores['rougeL'].recall
        results[params + '_rougeL_fmeasure'] = scores['rougeL'].fmeasure

    generations.append(results)

going for generation 0
k_1
To resolve the error you are experiencing, you should follow these steps:

1. Check the IP Address mentioned in the URL provided: https://launchpad.support.sap.com/#/notes/2395508. Make sure that this IP Address is allowed by your destination server. You may need to contact your IT department for more information on how to allowlist the IP Address.

2. Verify that the Operations URL 'https//.../sap.odm.finance.costobject.costcenter' is correct and accessible. Ensure that there are no typos or missing characters in the URL.

3. Check if there are any response options available for the HTTP Error Response code 404. This error typically indicates that the requested resource was not found. You may need to consult the documentation or contact SAP support for further assistance on how to handle this specific error.

If you are still unable to resolve the error after following these steps, it is recommended to reach out to SAP support for further assistance.
k_2
Bas

To resolve the error, you should follow these steps:

1. Check if the IP address mentioned in the SAP note 2395508 is allowed by your destination server. Contact your IT department to verify this information.

2. Make sure that the URL 'https//.../sap.odm.finance.costobject.costcenter' is correct and accessible. Check for any typos or missing characters in the URL.

3. Verify if the destination server is up and running. You can try accessing the URL in a web browser to see if it returns a valid response.

4. If the destination server is accessible, but you still receive a 404 error, contact SAP support for further assistance. Provide them with the error message and any relevant details for troubleshooting.

It's important to involve your IT department and SAP support to ensure a proper resolution to the error.
k_10
To resolve the error, you should follow these steps:

1. Check if the IP address mentioned in the SAP note 2395508 is allowed by your destination server. Contact your IT dep

Based on the error message you provided, it seems that there is an issue with connecting to the destination server. Here are the steps you can follow to resolve this error:

1. Check the IP Address mentioned in the provided URL (https://launchpad.support.sap.com/#/notes/2395508) and ensure that it is allowlisted by your destination server. You may need to contact your IT department for assistance with this.

2. Verify that the Operations URL ('https//.../sap.odm.finance.costobject.costcenter') is correct and accessible. Make sure there are no typos or missing characters in the URL.

3. If the URL is correct and accessible, check if the destination server is up and running. You can try accessing the URL in a web browser to see if it returns any response.

4. If the destination server is not accessible or returns a 404 error, it indicates that the requested resource is not found. In this case, you may need to review the configuration of the destination server and ensure that the necessar

To resolve the error, you should follow these steps:

1. Check if the IP address mentioned in the SAP Note 2395508 is allowed by your destination server. Contact your IT department to verify if the IP address is whitelisted.

2. Verify the URL 'https//.../sap.odm.finance.costobject.costcenter' and ensure that it is correct. Make sure there are no typos or missing characters in the URL.

3. Check if the destination server is accessible and running. You can try accessing the URL directly in a web browser to see if it returns any response. If it doesn't, there may be an issue with the server.

4. If the destination server is accessible, but you still receive a 404 error, it means that the requested resource (in this case, the cost center) was not found. Double-check the configuration and mapping of the cost center in SAP Master Data Integration to ensure it is set up correctly.

5. If you are unable to resolve the issue, consider reaching out to SAP Support for further assistance. Provide

To resolve the error, you should follow these steps:

1. Check if the IP Address mentioned in the SAP Note 2395508 is allowlisted by your destination server. Contact your IT department for more information on allowlisting IP addresses.

2. Verify the URL mentioned in the error message: 'https//.../sap.odm.finance.costobject.costcenter'. Make sure the URL is correct and accessible.

3. Check if the destination server is up and running. You can try accessing the URL directly in a web browser to see if it returns any response.

4. If the destination server is accessible and the URL is correct, but you still receive a 404 error, contact SAP support for further assistance. Provide them with the error message and any relevant details for troubleshooting.
k_7
To resolve the error, you should follow these steps:

1. Check if the IP address mentioned in the SAP Note 2395508 is allowed by your destination server. Contact your IT department for more information on allowlisting IP addresses.

2. V

In [ ]:
!say done

In [34]:
pd.DataFrame(generations).to_csv('results/results_k_rouge.csv')

In [35]:
rouge1_precision_scores = {}
rouge1_recall_scores = {}
rouge1_fmeasure_scores = {}
rougeL_precision_scores = {}
rougeL_recall_scores = {}
rougeL_fmeasure_scores = {}

# all_scoring = [rouge1_precision_scores, rouge1_recall_scores, rouge1_fmeasure_scores, rougeL_precision_scores, rougeL_recall_scores, rougeL_fmeasure_scores]
# all_scoring_str = ['rouge1_precision', 'rouge1_recall', 'rouge1_fmeasure', 'rougeL_precision', 'rougeL_recall', 'rougeL_fmeasure']

all_scoring = [rouge1_fmeasure_scores, rougeL_fmeasure_scores]
all_scoring_str = ['rouge1_fmeasure', 'rougeL_fmeasure']

for generation in generations:
    for column in generation:
        for i in range(len(all_scoring)):
            if all_scoring_str[i] in column:
                score = generation[column]
                if score in all_scoring[i]:
                    all_scoring[i][score].append(column)
                else:
                    all_scoring[i][score] = [column]

for scoring in all_scoring:
    sorted_scores = sorted(scoring)[::-1]
    for i in range(1, 10):
        print(scoring[sorted_scores[i]], sorted_scores[i])
    print()

['k_2_rouge1_fmeasure'] 0.4074074074074074
['k_14_rouge1_fmeasure'] 0.3982300884955753
['k_13_rouge1_fmeasure', 'k_13_rouge1_fmeasure', 'k_13_rouge1_fmeasure'] 0.39215686274509803
['k_2_rouge1_fmeasure', 'k_2_rouge1_fmeasure'] 0.3835616438356164
['k_15_rouge1_fmeasure'] 0.3801652892561983
['k_1_rouge1_fmeasure'] 0.3798449612403101
['k_8_rouge1_fmeasure', 'k_8_rouge1_fmeasure'] 0.3745819397993311
['k_1_rouge1_fmeasure', 'k_1_rouge1_fmeasure'] 0.37209302325581395
['k_4_rouge1_fmeasure'] 0.36423841059602646

['k_2_rougeL_fmeasure'] 0.23148148148148148
['k_14_rougeL_fmeasure'] 0.22123893805309736
['k_15_rougeL_fmeasure'] 0.2171945701357466
['k_4_rougeL_fmeasure'] 0.2119205298013245
['k_15_rougeL_fmeasure'] 0.2066115702479339
['k_10_rougeL_fmeasure', 'k_10_rougeL_fmeasure', 'k_10_rougeL_fmeasure'] 0.20535714285714285
['k_9_rougeL_fmeasure', 'k_9_rougeL_fmeasure', 'k_9_rougeL_fmeasure'] 0.20168067226890757
['k_1_rougeL_fmeasure', 'k_1_rougeL_fmeasure', 'k_1_rougeL_fmeasure'] 0.20155038759689

### Chunking rankings

['cs_600|co_60_rouge1_fmeasure', 'cs_600|co_60_rouge1_fmeasure'] 0.4474885844748858
['cs_600|co_150_rouge1_fmeasure', 'cs_600|co_150_rouge1_fmeasure', 'cs_600|co_150_rouge1_fmeasure'] 0.4085106382978724
['cs_200|co_100_rouge1_fmeasure', 'cs_200|co_100_rouge1_fmeasure'] 0.40466926070038906
['cs_400|co_40_rouge1_fmeasure', 'cs_400|co_40_rouge1_fmeasure', 'cs_400|co_40_rouge1_fmeasure'] 0.4
['cs_1400|co_20_rouge1_fmeasure'] 0.39676113360323884
['cs_200|co_60_rouge1_fmeasure', 'cs_200|co_60_rouge1_fmeasure', 'cs_200|co_60_rouge1_fmeasure'] 0.3941605839416058
['cs_400|co_60_rouge1_fmeasure', 'cs_400|co_60_rouge1_fmeasure'] 0.38938053097345127
['cs_1000|co_100_rouge1_fmeasure'] 0.3805970149253731
['cs_800|co_20_rouge1_fmeasure'] 0.37414965986394555

['cs_600|co_60_rougeL_fmeasure', 'cs_600|co_60_rougeL_fmeasure'] 0.27397260273972607
['cs_1200|co_40_rougeL_fmeasure', 'cs_1200|co_40_rougeL_fmeasure', 'cs_1200|co_40_rougeL_fmeasure'] 0.21926910299003322
['cs_1000|co_300_rougeL_fmeasure'] 0.21875
['cs_400|co_40_rougeL_fmeasure', 'cs_400|co_40_rougeL_fmeasure', 'cs_400|co_40_rougeL_fmeasure'] 0.21481481481481485
['cs_1000|co_300_rougeL_fmeasure', 'cs_1000|co_300_rougeL_fmeasure'] 0.2123287671232877
['cs_1200|co_150_rougeL_fmeasure', 'cs_1200|co_150_rougeL_fmeasure', 'cs_1400|co_20_rougeL_fmeasure', 'cs_1200|co_150_rougeL_fmeasure'] 0.21052631578947367
['cs_200|co_100_rougeL_fmeasure', 'cs_200|co_100_rougeL_fmeasure'] 0.2101167315175097
['cs_800|co_20_rougeL_fmeasure'] 0.20875420875420875
['cs_1400|co_150_rougeL_fmeasure'] 0.2080536912751678

### K rankings

['k_2_rouge1_fmeasure'] 0.4074074074074074
['k_14_rouge1_fmeasure'] 0.3982300884955753
['k_13_rouge1_fmeasure', 'k_13_rouge1_fmeasure', 'k_13_rouge1_fmeasure'] 0.39215686274509803
['k_2_rouge1_fmeasure', 'k_2_rouge1_fmeasure'] 0.3835616438356164
['k_15_rouge1_fmeasure'] 0.3801652892561983
['k_1_rouge1_fmeasure'] 0.3798449612403101
['k_8_rouge1_fmeasure', 'k_8_rouge1_fmeasure'] 0.3745819397993311
['k_1_rouge1_fmeasure', 'k_1_rouge1_fmeasure'] 0.37209302325581395
['k_4_rouge1_fmeasure'] 0.36423841059602646

['k_2_rougeL_fmeasure'] 0.23148148148148148
['k_14_rougeL_fmeasure'] 0.22123893805309736
['k_15_rougeL_fmeasure'] 0.2171945701357466
['k_4_rougeL_fmeasure'] 0.2119205298013245
['k_15_rougeL_fmeasure'] 0.2066115702479339
['k_10_rougeL_fmeasure', 'k_10_rougeL_fmeasure', 'k_10_rougeL_fmeasure'] 0.20535714285714285
['k_9_rougeL_fmeasure', 'k_9_rougeL_fmeasure', 'k_9_rougeL_fmeasure'] 0.20168067226890757
['k_1_rougeL_fmeasure', 'k_1_rougeL_fmeasure', 'k_1_rougeL_fmeasure'] 0.20155038759689922
['k_2_rougeL_fmeasure', 'k_2_rougeL_fmeasure'] 0.19863013698630136

### look more into K variation
### look for autoML equivalents for LLM / GenAI (perhaps in SSDL) - look if teams have found best practices | roundtable

#### mechanism to document; "ADR" parameter decision records for explainability - why / how was a certain set of parameters preferred over another | also roundtable
### validate parameter decisions with work groups / cross product architecture groups

In [169]:
query = "What is the first album by Sleep Token?"

docs = VectorStore.similarity_search(query=query, k=5)

chain = load_qa_chain(llm=all_llms['falcon-40b-instruct'], chain_type="stuff")
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=query)
    print(cb)
print(response)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
 The first album by Sleep Token is called "One".


In [51]:
llm_list = """`text-davinci-003` | `.openai.Completion(deployment_id='text-davinci-003')` | `.BTPOpenAI(deployment_id='text-davinci-003')` |
| `gpt-35-turbo` | `.openai.ChatCompletion(deployment_id='gpt-35-turbo')` | `.ChatOpenAI(deployment_id='gpt-35-turbo')` |
| `gpt-4` | `.openai.ChatCompletion(deployment_id='gpt-4')` | `.ChatOpenAI(deployment_id='gpt-4')` |
| `gpt-4-32k` | `.openai.ChatCompletion(deployment_id='gpt-4-32k')` | `.ChatOpenAI(deployment_id='gpt-4-32k')` |
| `alephalpha` | `.aleph_alpha.Client(deployment_id='alephalpha')` | `.AlephAlpha(deployment_id='alephalpha')` |
| `anthropic-claude-v1` |  | `.BedrockChat(deployment_id='anthropic-claude-v1')` |
| `anthropic-claude-v2` |  | `.BedrockChat(deployment_id='anthropic-claude-v2')` |
| `anthropic-claude-instant-v1` |  | `.BedrockChat(deployment_id='anthropic-claude-instant-v1')` |
| `anthropic-claude-v1-100k` |  | `.BedrockChat(deployment_id='anthropic-claude-v1-100k')` |
| `anthropic-claude-v2-100k` |  | `.BedrockChat(deployment_id='anthropic-claude-v2-100k')` |
| `anthropic-direct-claude-instant-1` |  | `.BedrockChat(deployment_id='anthropic-direct', model_kwargs={'model': 'claude-instant-1'})` |
| `anthropic-direct-claude-2` |  | `.BedrockChat(deployment_id='anthropic-direct', model_kwargs={'model': 'claude-2'})` |
| `ai21-j2-grande-instruct` |  | `.Bedrock(deployment_id='ai21-j2-grande-instruct')` |
| `ai21-j2-jumbo-instruct` |  | `.Bedrock(deployment_id='ai21-j2-jumbo-instruct')` |
| `amazon-titan-tg1-large` |  | `.Bedrock(deployment_id='amazon-titan-tg1-large')` |
| `gcp-text-bison-001` |  | `.GooglePalm(deployment_id='gcp-text-bison-001')` |
| `falcon-7b` | `.huggingface.Client(deployment_id='falcon-7b')` | `.HuggingFaceTextGenInference(deployment_id='falcon-7b')` |
| `falcon-40b-instruct` | `.huggingface.Client(deployment_id='falcon-40b-instruct')` | `.HuggingFaceTextGenInference(deployment_id='falcon-40b-instruct')` |
| `llama2-13b-chat-hf` | `.huggingface.Client(deployment_id='llama2-13b-chat-hf')` | `.HuggingFaceTextGenInference(deployment_id='llama2-13b-chat-hf')` |"""

In [38]:
sorted_rouge1_precision_scores = sorted(rouge1_precision_scores)[::-1]

for i in range(1, 4):
    print(rouge1_precision_scores[sorted_rouge1_precision_scores[i]], sorted_rouge1_precision_scores[i])

['ai21-j2-grande-instruct_rouge1_precision'] 0.47058823529411764
['gpt-4-32k_rouge1_precision'] 0.4642857142857143
['ai21-j2-jumbo-instruct_rouge1_precision'] 0.4375


In [30]:
rouge1_precision_scores

{0.2876712328767123: ['text-davinci-003_rouge1_precision'],
 0.281437125748503: ['gpt-35-turbo_rouge1_precision'],
 0.40540540540540543: ['gpt-4_rouge1_precision'],
 0.375: ['gpt-4-32k_rouge1_precision'],
 0.25: ['anthropic-claude-v1_rouge1_precision',
  'ai21-j2-jumbo-instruct_rouge1_precision'],
 0.23809523809523808: ['anthropic-claude-v2_rouge1_precision'],
 0.2696078431372549: ['anthropic-claude-instant-v1_rouge1_precision'],
 0.23346303501945526: ['anthropic-claude-v1-100k_rouge1_precision'],
 0.2777777777777778: ['anthropic-claude-v2-100k_rouge1_precision'],
 0.23577235772357724: ['anthropic-direct-claude-instant-1_rouge1_precision'],
 0.2422907488986784: ['anthropic-direct-claude-2_rouge1_precision'],
 0.23553719008264462: ['ai21-j2-grande-instruct_rouge1_precision'],
 0.4375: ['ai21-j2-jumbo-instruct_rouge1_precision'],
 0.24731182795698925: ['amazon-titan-tg1-large_rouge1_precision'],
 0.37037037037037035: ['gcp-text-bison-001_rouge1_precision'],
 1.0: ['falcon-7b_rouge1_preci

In [52]:
for row in llm_list.split('|\n|'):
    row_str = row.split('|')
    print(row_str[0].replace('`', '\'').strip() + ": ", row_str[2].replace('`', '').strip()[1:] + ",")

'text-davinci-003':  BTPOpenAI(deployment_id='text-davinci-003'),
'gpt-35-turbo':  ChatOpenAI(deployment_id='gpt-35-turbo'),
'gpt-4':  ChatOpenAI(deployment_id='gpt-4'),
'gpt-4-32k':  ChatOpenAI(deployment_id='gpt-4-32k'),
'alephalpha':  AlephAlpha(deployment_id='alephalpha'),
'anthropic-claude-v1':  BedrockChat(deployment_id='anthropic-claude-v1'),
'anthropic-claude-v2':  BedrockChat(deployment_id='anthropic-claude-v2'),
'anthropic-claude-instant-v1':  BedrockChat(deployment_id='anthropic-claude-instant-v1'),
'anthropic-claude-v1-100k':  BedrockChat(deployment_id='anthropic-claude-v1-100k'),
'anthropic-claude-v2-100k':  BedrockChat(deployment_id='anthropic-claude-v2-100k'),
'anthropic-direct-claude-instant-1':  BedrockChat(deployment_id='anthropic-direct', model_kwargs={'model': 'claude-instant-1'}),
'anthropic-direct-claude-2':  BedrockChat(deployment_id='anthropic-direct', model_kwargs={'model': 'claude-2'}),
'ai21-j2-grande-instruct':  Bedrock(deployment_id='ai21-j2-grande-instruct

### Quick legend
sourced from this folder: https://github.tools.sap/AI-Playground-Projects/llm-commons/tree/26b5e3972e56e16f0cfd2d373f2b7b53f8ad40b4/llm_commons/langchain/proxy  
* 'anthropic-claude-v1', 'anthropic-claude-instant-v1', 'anthropic-claude-v2', 'anthropic-claude-v1-100k', 'anthropic-claude-v2-100k' use "max_tokens_to_sample"  
* 'ai21-j2-grande-instruct', 'ai21-j2-jumbo-instruct' use "maxTokens" 


#### max_tokens_to_sample, maxTokens, max_output_tokens, max_new_tokens and of course "max_tokens" itself

In [61]:
emb_list = """ `text-embedding-ada-002-v2` | `.openai.Embedding(deployment_id='text-embedding-ada-002-v2')` | `.OpenAIEmbeddings(deployment_id='text-embedding-ada-002-v2')` |
| `amazon-titan-e1t-medium` |  | `.BedrockEmbeddings(deployment_id='amazon-titan-e1t-medium')` |
| `gcp-textembedding-gecko-001` |  | `.GoogleEmbeddings(deployment_id='gcp-textembedding-gecko-001')` |"""

In [66]:
for row in emb_list.split('|\n|'):
    row_str = row.split('|')
    print(row_str[0].replace('`', '\'').strip() + ": ", row_str[2].replace('`', '').strip()[1:] + ",")

'text-embedding-ada-002-v2':  OpenAIEmbeddings(deployment_id='text-embedding-ada-002-v2'),
'amazon-titan-e1t-medium':  BedrockEmbeddings(deployment_id='amazon-titan-e1t-medium'),
'gcp-textembedding-gecko-001':  GoogleEmbeddings(deployment_id='gcp-textembedding-gecko-001'),


### Check preview models

In [ ]:
all_playground_models = [
    
]

In [10]:
# txt version

raw_documents = TextLoader('corpus/txt/Sleep_Token.txt').load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
chunks = text_splitter.split_documents(raw_documents)

# DONE: try amazon titan and gcp vertex embeddings # update: they do not have vertex but palm bison
embeddings = BTPOpenAIEmbeddings(deployment_id="text-embedding-ada-002-v2")
VectorStore = FAISS.from_documents(chunks, embedding=embeddings)

In [11]:
example = """
Problem descr 

Costcenter replication (Masterdata integration) Company association - https://itsm.services.sap/now/cwf/agent/record/sn_customerservice_case/2116e080974b3d18ff3339371153af20

we need to map the Legal Entity to the Cost Center association in SF.  (see screenshot)

We do not find any information on this setting.


We don't have any problem about company/legal entity association.
Our problem is association integration for Cost Center Object via MDI Integration. Associations are not included to the standard field mappings between MDI and SF. On the Cost Center MDI admin guide, there isn't any part related to association field mappings. Is there a way to integrate cost center associations? We couldn't find any KBA and wanted to make sure about it.

https://help.sap.com/docs/SAP_SUCCESSFACTORS_EMPLOYEE_CENTRAL/634eabb3d94044d2b319aaf7a8f18fb9/75ace5cbc09645b58c9d2b2832918828.html?q=cost%20center%20legal%20entity


Analysis 

links 
https://me.sap.com/notes/2287729
https://me.sap.com/notes/2151132
https://me.sap.com/notes/2315012
https://me.sap.com/notes/2511175

https://help.sap.com/docs/SAP_SUCCESSFACTORS_EMPLOYEE_CENTRAL/634eabb3d94044d2b319aaf7a8f18fb9/75ace5cbc09645b58c9d2b2832918828.html?q=cost%20center%20legal%20entity
https://help.sap.com/docs/SAP_SUCCESSFACTORS_EMPLOYEE_CENTRAL/634eabb3d94044d2b319aaf7a8f18fb9/981db612c44a4768a121d1b033c45ed9.html?locale=en-US


Custom association to Legal Entity is not supported by MDI. Only custom fields are supported which reference Legal Entity.

I am sorry for the bad news, however this is confirmed that this is expected behaviour.

Based on that I set this ticket to soluton provided.

summary : The root cause is that custom association to Legal Entity is not supported by MDI. Only custom fields are supported which reference Legal Entity. This is confirmed as expected behavior, so the ticket is set to solution provided. The standard field mappings between MDI and SF do not include associations, and there is no part related to association field mappings in the Cost Center MDI admin guide. Therefore, there is no way to integrate cost center associations. The links provided do not contain any information on this setting, and there are no KBAs available for this issue.
"""